In [1]:
!pip install -q "transformers>=4.51.3" peft sentence-transformers faiss-cpu accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 39.2 MB/s eta 0:00:00


In [2]:
#Huggingface'e bağlanma
from huggingface_hub import login
login()

In [3]:
import os, torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

!git clone https://github.com/guzelTemli/YapayZekaChatbot.git

BASE_REPO = "/content/YapayZekaChatbot"
os.listdir(BASE_REPO)


Cloning into 'YapayZekaChatbot'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 290 (delta 9), reused 18 (delta 5), pack-reused 268 (from 2)
Receiving objects: 100% (290/290), 573.88 MiB | 16.86 MiB/s, done.
Resolving deltas: 100% (144/144), done.


['.gitattributes',
 'FINE-TUNE',
 'RAG',
 'README.md',
 '.git',
 'bilgi_guvenligi_dataset']

In [4]:
import pandas as pd
import numpy as np
import faiss

RAG_DIR = os.path.join(BASE_REPO, "RAG")

CHUNKS_META_PATH = os.path.join(RAG_DIR, "chunks_meta.csv")
INDEX_PATH       = os.path.join(RAG_DIR, "faiss_index.bin")
CHUNK_IDS_PATH   = os.path.join(RAG_DIR, "chunk_ids.npy")

# RAG meta (faiss ile kullanılan sade dosya)
chunks_df = pd.read_csv(CHUNKS_META_PATH)

# RAG index & id map
index     = faiss.read_index(INDEX_PATH)
chunk_ids = np.load(CHUNK_IDS_PATH)

print("chunks_meta satır sayısı:", len(chunks_df))
print("FAISS vektör sayısı:", index.ntotal)
print("chunk_ids şekli:", chunk_ids.shape)

chunks_df.head()


chunks_meta satır sayısı: 247
FAISS vektör sayısı: 247
chunk_ids şekli: (247,)


,chunk_id,filename,section_label,chunk_index,text
0,0,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.1,0,5.1 Bilgi güvenliğine ilişkin politikalar Bilg...
1,1,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.10,0,5.10 Bilginin ve diğer ilgili varlıkların kabu...
2,2,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.11,0,5.11 Varlıkların İadesi Personel ve diğer ilgi...
3,3,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.12,0,"5.12 Bilgilerin Sınıflandırılması Bilgiler, ku..."
4,4,ISO-27001-Controls-List-Free-Download-pdf-.pdf,ISO_5.13,0,5.13 Bilgilerin Etiketlenmesi Bilgi etiketleme...


In [5]:
MASTER_PATH = os.path.join(
    BASE_REPO,
    "bilgi_guvenligi_dataset",
    "bilgi_guvenligi_chunks_v3_tr_FOR_RAG_C_FINAL.csv"
)

master_df = pd.read_csv(MASTER_PATH)
print("master_df satır sayısı:", len(master_df))
master_df.head(), master_df.columns


master_df satır sayısı: 247


(                                         filename  \
 0  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 1  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 2  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 3  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 4  ISO-27001-Controls-List-Free-Download-pdf-.pdf   
 
                                           source_url section_label  \
 0  https://hightable.io/iso-27001-annex-a-control...       ISO_5.1   
 1  https://hightable.io/iso-27001-annex-a-control...      ISO_5.10   
 2  https://hightable.io/iso-27001-annex-a-control...      ISO_5.11   
 3  https://hightable.io/iso-27001-annex-a-control...      ISO_5.12   
 4  https://hightable.io/iso-27001-annex-a-control...      ISO_5.13   
 
    section_index  chunk_index  \
 0           5.10            0   
 1           5.10            0   
 2           5.11            0   
 3           5.12            0   
 4           5.13            0   
 
                                        

In [6]:
URL_MAP = {}

for _, row in master_df.iterrows():
    key = (row["filename"], row["section_label"], int(row["chunk_index"]))
    URL_MAP[key] = row["source_url"]

print("URL_MAP boyutu:", len(URL_MAP))
list(URL_MAP.items())[:3]


URL_MAP boyutu: 247


[(('ISO-27001-Controls-List-Free-Download-pdf-.pdf', 'ISO_5.1', 0),
  'https://hightable.io/iso-27001-annex-a-controls-list/'),
 (('ISO-27001-Controls-List-Free-Download-pdf-.pdf', 'ISO_5.10', 0),
  'https://hightable.io/iso-27001-annex-a-controls-list/'),
 (('ISO-27001-Controls-List-Free-Download-pdf-.pdf', 'ISO_5.11', 0),
  'https://hightable.io/iso-27001-annex-a-controls-list/')]

In [7]:
from sentence_transformers import SentenceTransformer

embed_model_name = "intfloat/multilingual-e5-large"  # RAG.ipynb'de ne kullandıysan aynısı
embed_model = SentenceTransformer(embed_model_name, device=device)
embed_model.max_seq_length = 512

embed_model


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False, 'architecture': 'XLMRobertaModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [9]:
TEXT_COL = "text"   # chunks_meta içindeki metin kolonu
RAG_MIN_SCORE = 0.80  # istersen ileride eşik olarak kullanırsın

def search_chunks(query: str, top_k: int = 5):
    """
    Hazır FAISS index + chunk_ids + chunks_meta.csv + master_df(URL_MAP)
    kullanarak soruya en çok benzeyen top_k chunk'ı döndürür.
    """
    query_text = f"query: {query}"

    with torch.no_grad():
        q_emb = embed_model.encode(
            [query_text],
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )

    scores, indices = index.search(q_emb, top_k)

    results = []
    for score, idx in zip(scores[0], indices[0]):
        cid = int(chunk_ids[idx])
        row = chunks_df[chunks_df["chunk_id"] == cid].iloc[0]

        filename      = row["filename"]
        section_label = row.get("section_label", "Genel")
        chunk_index   = int(row["chunk_index"])
        text          = row[TEXT_COL]

        # URL'yi master_df'ten çek
        key = (filename, section_label, chunk_index)
        url = URL_MAP.get(key, None)

        results.append({
            "score": float(score),
            "chunk_id": cid,
            "chunk_index": chunk_index,
            "section_label": section_label,
            "filename": filename,
            "text": text,
            "url": url
        })
    return results


In [10]:
test_res = search_chunks("Uzaktan çalışma sırasında VPN neden zorunlu tutulmalıdır?", top_k=3)
for i, ch in enumerate(test_res, 1):
    print(f"[{i}] {ch['filename']} | {ch['section_label']} | idx={ch['chunk_index']} | skor={ch['score']:.3f}")
    print("URL:", ch['url'])
    print("Metin:", ch['text'][:120], "...\n")


[1] ISO-27001-Controls-List-Free-Download-pdf-.pdf | ISO_6.7 | idx=0 | skor=0.865
URL: https://hightable.io/iso-27001-annex-a-controls-list/
Metin: 6.7 Uzaktan çalışma Personel, kuruluşun tesisleri dışında erişilen, işlenen veya saklanan bilgileri korumak için uzaktan ...

[2] SANS_Acceptable_Encryption_Standard_April2025.pdf | full | idx=3 | skor=0.844
URL: https://www.sans.org/information-security-policy/acceptable-encryption-standard/
Metin: ve uzaktan erişim ayrıcalıklarının geçici olarak askıya alınmasına ve ciddi durumlarda iş akdinin veya sözleşmeden doğan ...

[3] SANS_Internal_Network_Access_Management_Policy_April2025.pdf | Safeguards | idx=1 | skor=0.838
URL: https://www.sans.org/information-security-policy/internal-network-access-management-policy/
Metin: (802.1x) kablosuz ağlarının her biri için sertifika tabanlı kimlik doğrulama gerektirir. INA-10 Kuruluşun kablosuz ağlar ...



In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL_ID = "google/gemma-2-2b-it"
ADAPTER_PATH  = os.path.join(BASE_REPO, "FINE-TUNE", "gemma-2b-train-qlora-final")

# Tokenizer (adapter klasöründen veya base modelden)
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# 1) Base model (saf Gemma)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
base_model.config.pad_token_id = tokenizer.eos_token_id
base_model.eval()

# 2) Fine-tune edilmiş model için ayrı bir base yükleyip LoRA giydiriyoruz
ft_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
ft_model = PeftModel.from_pretrained(
    ft_base,
    ADAPTER_PATH,
)
ft_model.config.pad_token_id = tokenizer.eos_token_id
ft_model.eval()

print("✅ Base ve fine-tune modeller yüklendi.")


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Base ve fine-tune modeller yüklendi.


In [12]:
def generate_det(model, tokenizer, prompt: str, max_new_tokens: int = 400):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,   # 🔒 deterministik
        )

    out_ids = outputs[0][inputs["input_ids"].shape[-1]:]
    text = tokenizer.decode(out_ids, skip_special_tokens=True).strip()
    return text


In [13]:
#Sadece soru (RAG yok)
def build_plain_prompt(question: str) -> str:
    user_content = (
        "Sen bilgi güvenliği ve ISO 27001 alanında soru yanıtlayan bir asistansın. "
        "Elindeki genel bilgiye göre, açık ve teknik ama anlaşılır Türkçe kullanarak cevap ver. "
        "Emin olmadığın konularda uydurma, kısıtlarını belirt.\n\n"
        f"Soru: {question}"
    )

    messages = [
        {"role": "user", "content": user_content}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    return prompt


In [14]:
#RAG’li içerik + prompt
def build_rag_user_content(question: str, retrieved_chunks) -> str:
    if not retrieved_chunks:
        context_text = "İlgili kaynak bulunamadı."
    else:
        blocks = []
        for i, ch in enumerate(retrieved_chunks, 1):
            block = (
                f"[{i}] Kaynak (dosya: {ch['filename']}, bölüm: {ch['section_label']}, chunk_index: {ch['chunk_index']})\n"
                f"{ch['text']}"
            )
            blocks.append(block)
        context_text = "\n\n".join(blocks)

    user_content = (
        "Sen ISO 27001 ve SANS bilgi güvenliği politikaları konusunda uzman bir asistansın. "
        "Aşağıda ilgili politika parçalarını göreceksin.\n\n"
        "Sadece bu kaynaklara dayanarak cevap ver. "
        "Eğer kaynaklar soruyu yanıtlamaya yetmiyorsa veya emin değilsen:\n"
        "- 'Kaynaklarda bu sorunun yanıtı yok.' de\n"
        "- KESİNLİKLE tahmin yürütme veya uydurma.\n\n"
        "=== KAYNAK METİNLER ===\n"
        f"{context_text}\n\n"
        "=== GÖREVİN ===\n"
        "1) Soruyu bu kaynaklara dayanarak yanıtla.\n"
        "2) Kaynaklarda olmayan bilgiyi uydurma, emin değilsen bunu açıkça söyle.\n\n"
        f"=== SORU ===\n{question}"
    )
    return user_content


def build_rag_prompt(question: str, top_k: int = 5):
    retrieved = search_chunks(question, top_k=top_k)
    user_content = build_rag_user_content(question, retrieved)

    messages = [
        {"role": "user", "content": user_content}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    return prompt, retrieved


In [15]:
def run_all_variants(question: str, top_k: int = 5):
    results = []

    # 1) Base model (no RAG)
    plain_prompt = build_plain_prompt(question)
    base_plain_answer = generate_det(base_model, tokenizer, plain_prompt)
    results.append({
        "variant": "A) Base model (no RAG)",
        "answer": base_plain_answer,
        "used_chunks": [],
        "source_links": []
    })

    # 2) Base model + RAG
    rag_prompt, retrieved_base = build_rag_prompt(question, top_k=top_k)
    base_rag_answer = generate_det(base_model, tokenizer, rag_prompt)
    base_links = sorted({ch["url"] for ch in retrieved_base if ch.get("url")})

    results.append({
        "variant": "B) Base model + RAG",
        "answer": base_rag_answer,
        "used_chunks": retrieved_base,
        "source_links": base_links
    })

    # 3) Fine-tuned model (no RAG)
    ft_plain_prompt = build_plain_prompt(question)
    ft_plain_answer = generate_det(ft_model, tokenizer, ft_plain_prompt)
    results.append({
        "variant": "C) Fine-tuned model (no RAG)",
        "answer": ft_plain_answer,
        "used_chunks": [],
        "source_links": []
    })

    # 4) Fine-tuned model + RAG
    ft_rag_prompt, retrieved_ft = build_rag_prompt(question, top_k=top_k)
    ft_rag_answer = generate_det(ft_model, tokenizer, ft_rag_prompt)
    ft_links = sorted({ch["url"] for ch in retrieved_ft if ch.get("url")})

    results.append({
        "variant": "D) Fine-tuned model + RAG",
        "answer": ft_rag_answer,
        "used_chunks": retrieved_ft,
        "source_links": ft_links
    })

    return results


In [16]:
import pandas as pd

questions = [
    # --- Kapsam içi + günlük dil ---
    "Hocam, uzaktan çalışırken neden illa VPN açıyoruz? Açmazsak ne olur?",
    "Canım asistanım, güçlü bir parola politikası tam olarak nasıl yazılır? Adım adım anlatır mısın?",
    "Ya şöyle sorayım; ayrıcalıklı hesapları neden bu kadar sıkı yönetiyoruz?",
    "Log’ları neden saklıyoruz hocam? Yani tam olarak ne işimize yarıyor?",

    # --- Gri alan ---
    "Hocam, şirketin sosyal medya hesaplarını yönetirken bilgi güvenliği açısından nelere dikkat edilmeli?",
    "Ya şimdi merak ettim; yapay zekâ sistemleri kurum güvenliğini nasıl etkiler?",
    "Evdeki modeme bağlı çalışanlar var, canım; onların Wi-Fi’yi nasıl yapılandırması gerekir?",

    # --- Kapsam dışı ---
    "Canım asistanım, kediler strese girince neden tırmalıyor?",
    "Hocam, bu yıl sonu dolar kaç TL olur sence?",
    "Ya sana bir şey sorcam; Türkiye’de en iyi tatlı hangisi?"
]

results_list = []

In [ ]:


for q in questions:
    print("ÇALIŞTIRILIYOR:", q)
    print("----------------------------------------------")

    comp = run_all_variants(q, top_k=4)

    for item in comp:
        results_list.append({
            "question": q,
            "variant": item["variant"],
            "answer": item["answer"],
            "sources": ", ".join(item["source_links"]) if item["source_links"] else "—"
        })

df_results = pd.DataFrame(results_list)
df_results.head()


ÇALIŞTIRILIYOR: Hocam, uzaktan çalışırken neden illa VPN açıyoruz? Açmazsak ne olur?
----------------------------------------------
ÇALIŞTIRILIYOR: Canım asistanım, güçlü bir parola politikası tam olarak nasıl yazılır? Adım adım anlatır mısın?
----------------------------------------------
ÇALIŞTIRILIYOR: Ya şöyle sorayım; ayrıcalıklı hesapları neden bu kadar sıkı yönetiyoruz?
----------------------------------------------
ÇALIŞTIRILIYOR: Log’ları neden saklıyoruz hocam? Yani tam olarak ne işimize yarıyor?
----------------------------------------------
ÇALIŞTIRILIYOR: Hocam, şirketin sosyal medya hesaplarını yönetirken bilgi güvenliği açısından nelere dikkat edilmeli?
----------------------------------------------
ÇALIŞTIRILIYOR: Ya şimdi merak ettim; yapay zekâ sistemleri kurum güvenliğini nasıl etkiler?
----------------------------------------------
ÇALIŞTIRILIYOR: Evdeki modeme bağlı çalışanlar var, canım; onların Wi-Fi’yi nasıl yapılandırması gerekir?
------------------------------

,question,variant,answer,sources
0,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",A) Base model (no RAG),## Uzaktan Çalışırken VPN'i Açmak Neden Önemli...,—
1,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",B) Base model + RAG,"Bu kaynaklar, uzaktan çalışırken VPN'i açmanın...",https://hightable.io/iso-27001-annex-a-control...
2,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",C) Fine-tuned model (no RAG),"VPN ile trafiğin şifrelenmesi sağlanır, üçüncü...",—
3,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",D) Fine-tuned model + RAG,"Uzaktan erişim için VPN kullanılması, kuruluşu...",https://hightable.io/iso-27001-annex-a-control...
4,"Canım asistanım, güçlü bir parola politikası t...",A) Base model (no RAG),"Elbette, güçlü bir parola politikası oluşturma...",—


In [ ]:
df_results.to_csv("rag_finetune_comparison_results.csv", index=False)
df_results


,question,variant,answer,sources
0,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",A) Base model (no RAG),## Uzaktan Çalışırken VPN'i Açmak Neden Önemli...,—
1,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",B) Base model + RAG,"Bu kaynaklar, uzaktan çalışırken VPN'i açmanın...",https://hightable.io/iso-27001-annex-a-control...
2,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",C) Fine-tuned model (no RAG),"VPN ile trafiğin şifrelenmesi sağlanır, üçüncü...",—
3,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",D) Fine-tuned model + RAG,"Uzaktan erişim için VPN kullanılması, kuruluşu...",https://hightable.io/iso-27001-annex-a-control...
4,"Canım asistanım, güçlü bir parola politikası t...",A) Base model (no RAG),"Elbette, güçlü bir parola politikası oluşturma...",—
5,"Canım asistanım, güçlü bir parola politikası t...",B) Base model + RAG,"Üzgünüm, ancak bu kaynaklar güçlü bir parola p...",https://www.sans.org/information-security-poli...
6,"Canım asistanım, güçlü bir parola politikası t...",C) Fine-tuned model (no RAG),Parola oluşturma sürecinde şu adımlar izlenmel...,—
7,"Canım asistanım, güçlü bir parola politikası t...",D) Fine-tuned model + RAG,"Şifreleme, farklı erişim seviyelerine ilişkin ...",https://www.sans.org/information-security-poli...
8,Ya şöyle sorayım; ayrıcalıklı hesapları neden ...,A) Base model (no RAG),## Açık ve Teknik Cevap:\n\n**İşte ayrıcalıklı...,—
9,Ya şöyle sorayım; ayrıcalıklı hesapları neden ...,B) Base model + RAG,"İşte sorunun cevabı, ISO 27001 ve SANS standar...",https://hightable.io/iso-27001-annex-a-control...


In [ ]:
df_results


,question,variant,answer,sources
0,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",A) Base model (no RAG),## Uzaktan Çalışırken VPN'i Açmak Neden Önemli...,—
1,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",B) Base model + RAG,"Bu kaynaklar, uzaktan çalışırken VPN'i açmanın...",https://hightable.io/iso-27001-annex-a-control...
2,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",C) Fine-tuned model (no RAG),"VPN ile trafiğin şifrelenmesi sağlanır, üçüncü...",—
3,"Hocam, uzaktan çalışırken neden illa VPN açıyo...",D) Fine-tuned model + RAG,"Uzaktan erişim için VPN kullanılması, kuruluşu...",https://hightable.io/iso-27001-annex-a-control...
4,"Canım asistanım, güçlü bir parola politikası t...",A) Base model (no RAG),"Elbette, güçlü bir parola politikası oluşturma...",—
5,"Canım asistanım, güçlü bir parola politikası t...",B) Base model + RAG,"Üzgünüm, ancak bu kaynaklar güçlü bir parola p...",https://www.sans.org/information-security-poli...
6,"Canım asistanım, güçlü bir parola politikası t...",C) Fine-tuned model (no RAG),Parola oluşturma sürecinde şu adımlar izlenmel...,—
7,"Canım asistanım, güçlü bir parola politikası t...",D) Fine-tuned model + RAG,"Şifreleme, farklı erişim seviyelerine ilişkin ...",https://www.sans.org/information-security-poli...
8,Ya şöyle sorayım; ayrıcalıklı hesapları neden ...,A) Base model (no RAG),## Açık ve Teknik Cevap:\n\n**İşte ayrıcalıklı...,—
9,Ya şöyle sorayım; ayrıcalıklı hesapları neden ...,B) Base model + RAG,"İşte sorunun cevabı, ISO 27001 ve SANS standar...",https://hightable.io/iso-27001-annex-a-control...


In [19]:
q = "Ya şimdi merak ettim; yapay zekâ sistemleri kurum güvenliğini nasıl etkiler?"
comp = run_all_variants(q, top_k=4)

for item in comp:
    print("====================================")
    print(item["variant"])
    print("------------------------------------")
    print(item["answer"])
    if item["source_links"]:
        print("\nKaynak linkleri:")
        for url in item["source_links"]:
            print(" -", url)
    print("\n")


A) Base model (no RAG)
------------------------------------
## Yapay Zeka ve Kurum Güvenliği: Bir İlişkisi

Yapay zeka (YZ) sistemleri, kurumların güvenliğini hem olumlu hem de olumsuz yönde etkileyebilecek potansiyel bir güçtür. 

**Olumlu Yönler:**

* **Güvenlik Sistemleri:** YZ, güvenlik sistemlerini daha etkili hale getirebilir. Örneğin, YZ algoritmaları, şüpheli aktiviteleri algılayabilir, saldırıların önüne geçebilir ve güvenlik sistemlerini daha hızlı ve etkili bir şekilde çalıştırabilir. 
* **Güvenlik Analizi:** YZ, büyük miktarda verileri analiz ederek, olası tehditleri tespit edebilir ve riskleri değerlendirebilir. Bu, kurumların daha iyi bir şekilde risk yönetimi stratejileri geliştirebilmesini sağlar.
* **Güvenlik Eğitimi:** YZ, güvenlik eğitimlerinde kullanılabilen bir araç olabilir. Örneğin, YZ, güvenlik eğitimlerinde gerçekçi ve etkileyici örnekler sunarak, kullanıcıların güvenlik risklerini daha iyi anlamalarını sağlayabilir.

**Olumsuz Yönler:**

* **Güvenlik Açığı:** 